# Collecte des données



Pour notre étude, nous avons entrepris la collecte de données à partir de deux sources distinctes : la base de données de Zekin et l'application interne Upay, utilisée pour faciliter les transactions financières. Ces deux sources ont été choisies pour garantir une couverture complète des données relatives aux transactions financières, offrant ainsi une vision globale et détaillée de notre domaine d'étude.

Les données recueillies ont été soigneusement exportées et organisées dans des fichiers au format Excel. Cette étape a permis de s'assurer de l'intégrité des données et de leur facilité d'utilisation pour notre analyse ultérieure.

L'analyse méticuleuse des informations recueillies a été une étape essentielle de notre processus. Nous avons mis l'accent sur la sélection judicieuse des colonnes pertinentes pour notre étude, en nous concentrant sur les données les plus significatives pour répondre à nos questions de recherche. Cette approche nous a permis de cibler les informations les plus importantes et de les interpréter de manière approfondie, contribuant ainsi à la qualité et à la pertinence de nos résultats.

En vue d'obtenir un ensemble de données consolidé et harmonieux, nous avons fusionné les données provenant de ces deux sources. Cette fusion nous a permis de combiner les informations complémentaires provenant de chaque source, offrant ainsi une vision plus complète et approfondie des transactions financières étudiées. Grâce à cette approche, nous avons pu établir des liens et des corrélations entre les différentes données, enrichissant ainsi notre analyse.

Dans le souci de préserver la confidentialité et la vie privée des individus concernés, une étape cruciale de notre processus a été l'anonymisation des données. Concrètement, nous avons remplacé les informations sensibles par des identifiants uniques plutôt que de conserver des détails identifiables tels que des noms ou des numéros d'identification personnels. Cette mesure vise à assurer la conformité aux normes éthiques et légales en matière de protection des données, démontrant ainsi notre engagement envers la protection de la vie privée et la confidentialité des individus impliqués dans notre étude.

Au final, notre dataset est constitué d'un ensemble de colonnes soigneusement définies pour enregistrer des détails cruciaux liés aux transactions de vote. Chaque transaction est unique et identifiable grâce à son "idTransaction", permettant une traçabilité précise. Le type de paiement associé à chaque transaction est consignée dans la colonne "process", tandis que le "keyWordSent" capture le mot clé spécifique lié à l'événement ou au nominé envoyé par le votant. Des informations telles que le pseudonyme du votant, le montant total de la transaction ("amount"), le nombre de votes accordés ("nberVote"), et le numéro de téléphone du votant sont également enregistrées. La colonne "status" reflète l'état actuel de la transaction, avec un code associé dans "statusCode" pour une compréhension plus détaillée. Pour une vue exhaustive, la diversité des canaux de vote, des pays, des événements culturels, et des catégories est consignée respectivement dans les colonnes "canal", "country", "event", et "category". Les détails techniques, tels que le navigateur utilisé ("browser"), l'adresse IP ("ip"), et l'agrégateur de paiement, complètent cette structure de collecte de données.

In [3]:
#!pip install pandas-profiling
#!pip install typing_extensions


In [4]:
# importation des biblithèques
import pandas as pd
import numpy as np

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from base64 import b64encode, b64decode

from sklearn.impute import SimpleImputer


from ydata_profiling import ProfileReport

In [5]:
# historique de vote sur Zekin
zekin_data = pd.read_excel('/content/drive/Othercomputers/Mon ordinateur/DIT MASTER1 2022/MASTER 2/MEMOIRE/data/data/zekin.story_voting.xlsx')

# historique de transaction sur upay
upay_data = pd.read_excel('/content/drive/Othercomputers/Mon ordinateur/DIT MASTER1 2022/MASTER 2/MEMOIRE/data/data/upay.transaction.xlsx')

In [6]:
zekin_data.columns

Index(['_id', 'process', 'keyWordSent', 'pseudo', 'amount', 'nberVote',
       'phoneNumber', 'idTransaction', 'refInterne', 'idExterne', 'refExterne',
       'status', 'statusCode', 'archive', 'createdAt', 'updatedAt', 'manager',
       'promoter', 'event', 'category', 'committee', 'percentage', 'tva',
       'tvaAmount', 'zekinAmount', 'canal', 'country', 'message', 'notifie',
       'bonus', 'agregator', 'email', 'laureate', 'sessionId', 'package',
       'browser', 'ip'],
      dtype='object')

In [7]:
upay_data.columns

Index(['id', 'service_id', 'description', 'id_transaction', 'ref_externe',
       'ref_interne', 'uid', 'amount', 'agregator', 'status', 'fees',
       'first_name', 'last_name', 'email', 'phone', 'currency', 'country',
       'process', 'created_at', 'updated_at', 'expired_at', 'env',
       'origin_amount', 'final_amount', 'currency_change', 'message',
       'phone_paie', 'call_back', 'web_hook', 'url_paie', 'extra_data', 'mode',
       'operator_details', 'init_by_id', 'short_url', 'checking', 'resend'],
      dtype='object')

In [8]:
# Fusionner les deux DataFrames sur les colonnes 'idtransaction' et 'id'
merged_df = pd.merge(zekin_data, upay_data, left_on='idTransaction', right_on='uid', how='left')

# Mettre à jour les valeurs de la colonne 'agregator_y' avec celles de 'agregator_x' lorsque 'idtransaction' correspond
merged_df['agregator'] = merged_df['agregator_y'].combine_first(merged_df['agregator_x'])

In [9]:
merged_df.columns

Index(['_id', 'process_x', 'keyWordSent', 'pseudo', 'amount_x', 'nberVote',
       'phoneNumber', 'idTransaction', 'refInterne', 'idExterne', 'refExterne',
       'status_x', 'statusCode', 'archive', 'createdAt', 'updatedAt',
       'manager', 'promoter', 'event', 'category', 'committee', 'percentage',
       'tva', 'tvaAmount', 'zekinAmount', 'canal', 'country_x', 'message_x',
       'notifie', 'bonus', 'agregator_x', 'email_x', 'laureate', 'sessionId',
       'package', 'browser', 'ip', 'id', 'service_id', 'description',
       'id_transaction', 'ref_externe', 'ref_interne', 'uid', 'amount_y',
       'agregator_y', 'status_y', 'fees', 'first_name', 'last_name', 'email_y',
       'phone', 'currency', 'country_y', 'process_y', 'created_at',
       'updated_at', 'expired_at', 'env', 'origin_amount', 'final_amount',
       'currency_change', 'message_y', 'phone_paie', 'call_back', 'web_hook',
       'url_paie', 'extra_data', 'mode', 'operator_details', 'init_by_id',
       'short_url', '

In [26]:
# Sélection des colonnes utiles
dataset = merged_df[['process_x', 'amount_x', 'phoneNumber', 'statusCode', 'createdAt', 'canal', 'country_x', 'event', 'browser', 'ip', 'agregator']]

In [27]:
# renommer les colonnes
dataset.columns = ['process' if col == 'process_x' else col for col in dataset.columns]
dataset.columns = ['amount' if col == 'amount_x' else col for col in dataset.columns]
dataset.columns = ['status' if col == 'status_x' else col for col in dataset.columns]
dataset.columns = ['country' if col == 'country_x' else col for col in dataset.columns]

# Nettoyage des données : Gestion des valeurs manquantes et correction des erreurs de saisie

Lorsque nous travaillons avec des ensembles de données, il est fréquent de rencontrer des valeurs manquantes ou des erreurs de saisie qui peuvent compromettre la qualité de nos analyses. Pour garantir la fiabilité de nos résultats, il est essentiel de nettoyer les données en traitant ces problèmes de manière appropriée.

*Traitement des valeurs manquantes :*

Les valeurs manquantes dans notre jeu de données surviennent pour diverses raisons, telles que des erreurs de collecte, des données non disponibles et des enregistrements incomplets. Pour maintenir l'intégrité de nos analyses, nous devons traiter ces valeurs manquantes de manière efficace. Dans notre démarche, nous avons opté pour des techniques d'imputation avancées, basées sur des modèles, afin d'estimer ces valeurs manquantes en fonction des autres caractéristiques de nos données.

Plutôt que de simplement supprimer les lignes contenant des valeurs manquantes, ce qui peut entraîner une perte d'informations précieuses, nous avons choisi d'utiliser des méthodes plus sophistiquées. En appliquant des modèles d'imputation, nous sommes en mesure d'estimer les valeurs manquantes en se basant sur les relations existantes entre les différentes variables de notre ensemble de données. Par exemple, nous pouvons utiliser des techniques telles que la régression linéaire, les k-plus proches voisins (KNN), ou encore des méthodes basées sur des algorithmes d'apprentissage automatique. En ce qui nous concerne, nous avons utilisé un algorithme d'apprentissage automatique, SimpleImputer de sklearn.impute, pour le traitement des valeurs manquantes.

En utilisant ces techniques d'imputation avancées, nous sommes en mesure de remplacer les valeurs manquantes par des valeurs estimées, ce qui nous permet de conserver l'intégrité de nos données tout en minimisant la perte d'informations. Cela nous permet d'obtenir des résultats plus précis et fiables lors de nos analyses ultérieures.

*Correction des erreurs de saisie :*

En plus de traiter les valeurs manquantes, nous avons également pris des mesures pour corriger les erreurs de saisie dans nos données. Ces erreurs proviennent de diverses sources, telles que des erreurs humaines lors de la saisie des données et des problèmes de formatage. Pour remédier à cela, nous avons utilisé des techniques de validation et de nettoyage des données, telles que la détection et la correction automatique des erreurs de typographie, la normalisation des valeurs et la vérification de la cohérence des données.

In [28]:
# visualisation des valeurs null
dataset.isnull().sum()

process            0
amount             0
phoneNumber        0
statusCode        65
createdAt          0
canal             19
country            0
event              0
browser        53041
ip             53041
agregator          0
dtype: int64

In [29]:
# Sélectionner uniquement les colonnes statusCode", "updatedAt", "browser", "ip" et "canal"
cols_to_impute = ["statusCode", "browser", "ip", "canal"]
data_to_impute = dataset[cols_to_impute]

# Instancier l'imputeur le plus fréquent
imputer = SimpleImputer(strategy="most_frequent")

# Effectuer l'imputation sur les colonnes sélectionnées
data_imputed = imputer.fit_transform(data_to_impute)

# Remplacer les valeurs imputées dans le jeu de données original
data_imputed_df = pd.DataFrame(data_imputed, columns=cols_to_impute)
dataset[cols_to_impute] = data_imputed_df



<ipython-input-29-cd8e6dc7f79f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[cols_to_impute] = data_imputed_df


In [30]:
# visualisation des valeurs null
dataset.isnull().sum()

process        0
amount         0
phoneNumber    0
statusCode     0
createdAt      0
canal          0
country        0
event          0
browser        0
ip             0
agregator      0
dtype: int64

In [31]:
dataset['canal'].unique()

array(['website', 'WhatsApp', 'Telegram', 'Website', 'Messenger',
       'whatsapp'], dtype=object)

In [32]:
# Correction automatique des erreurs de typographie dans la colonne canal
def correct_typo(column):
    corrections = {
        'website': 'Website',
        'whatsapp': 'WhatsApp',
        'Whatsapp': 'WhatsApp',
    }
    return column.replace(corrections)

dataset['canal'] = correct_typo(dataset['canal'])


<ipython-input-32-f9dfde08ce98>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['canal'] = correct_typo(dataset['canal'])


In [33]:
dataset['canal'].unique()

array(['Website', 'WhatsApp', 'Telegram', 'Messenger'], dtype=object)

In [34]:
dataset.to_excel('/content/drive/Othercomputers/Mon ordinateur/DIT MASTER1 2022/MASTER 2/MEMOIRE/data/data/dataset_cleaned_12022024.xlsx', index=False)

# Pseudonymisation des numéros de téléphone et des IP

La pseudonymisation des numéros de téléphone est le processus de substitution ou de modification des données de manière à ce qu'elles ne puissent plus être directement liées à une personne identifiable sans l'utilisation d'informations supplémentaires. Cela vise à protéger la vie privée des individus tout en permettant l'utilisation des données à des fins légitimes.

Voici quelques méthodes courantes de pseudonymisation des numéros de téléphone :

1. **Hachage :** Il s'agit de convertir le numéro de téléphone en une chaîne de caractères aléatoire de longueur fixe à l'aide d'une fonction de hachage. Cependant, cette méthode est irréversible, ce qui signifie qu'il n'est pas possible de récupérer le numéro de téléphone d'origine à partir du haché.

2. **Tokenisation :** Les numéros de téléphone peuvent être remplacés par des jetons aléatoires. Une table de correspondance est utilisée pour relier les jetons aux numéros de téléphone d'origine, mais cette table est stockée de manière sécurisée et n'est accessible qu'aux personnes autorisées.

3. **Anonymisation partielle :** Cette méthode consiste à masquer une partie du numéro de téléphone tout en en préservant une partie pour des fins d'identification. Par exemple, les quatre derniers chiffres peuvent être masqués, laissant les six premiers visibles.

4. **Génération de pseudonyme aléatoire :** Les numéros de téléphone peuvent être remplacés par des identifiants ou des pseudonymes aléatoires qui ne sont pas directement liés à l'individu réel.

5. **Chiffrement :** Les numéros de téléphone peuvent être chiffrés à l'aide d'un algorithme de chiffrement, de sorte qu'ils ne puissent être déchiffrés que par des personnes disposant de la clé appropriée.

Il est important de noter que la pseudonymisation ne garantit pas une sécurité totale, et la protection des données dépend également de la manière dont les données pseudonymisées sont stockées, traitées et qui y a accès.

Dans notre cas, nous avons choisi d'utiliser le chiffrement. Nous nous sommes assurés de bien sécuriser la clé de chiffrement, car elle est cruciale pour le processus de déchiffrement et pour la confidentialité des données.

In [ ]:
def encrypt_phone_number(phone_number, key="VWHtrxyx^5jmZvp!dHMFNWv@v#v17c^X"):
    # Convertir la clé en format conforme à l'algorithme AES
    key = key.ljust(32)[:32].encode('utf-8')

    # Initialiser le chiffreur AES avec le mode CBC (Cipher Block Chaining)
    cipher = Cipher(algorithms.AES(key), modes.CFB(b'\0' * 16), backend=default_backend())
    encryptor = cipher.encryptor()

    # Ajouter un padding aux données avant le chiffrement
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(phone_number.encode('utf-8')) + padder.finalize()

    # Chiffrer les données
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()

    # Retourner le texte chiffré en base64 pour un stockage sûr
    return b64encode(ciphertext).decode('utf-8')

def decrypt_phone_number(encrypted_phone_number, key):
    # Convertir la clé en format conforme à l'algorithme AES
    key = key.ljust(32)[:32].encode('utf-8')

    # Initialiser le déchiffreur AES avec le mode CBC (Cipher Block Chaining)
    cipher = Cipher(algorithms.AES(key), modes.CFB(b'\0' * 16), backend=default_backend())
    decryptor = cipher.decryptor()

    # Décoder le texte chiffré en base64
    ciphertext = b64decode(encrypted_phone_number.encode('utf-8'))

    # Déchiffrer les données
    decrypted_data = decryptor.update(ciphertext) + decryptor.finalize()

    # Retirer le padding ajouté avant le chiffrement
    unpadder = padding.PKCS7(128).unpadder()
    phone_number = unpadder.update(decrypted_data) + unpadder.finalize()

    # Retourner le numéro de téléphone décrypté
    return phone_number.decode('utf-8')

In [ ]:
# Appliquer le chiffrement sur la colonne contenant les numéros de téléphone
dataset['phoneNumber'] = dataset['phoneNumber'].apply(lambda x: encrypt_phone_number(str(x)))

<ipython-input-121-20ac6486058c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['phoneNumber'] = dataset['phoneNumber'].apply(lambda x: encrypt_phone_number(str(x)))


In [ ]:
# Appliquer le chiffrement sur la colonne contenant les numéros de téléphone
dataset['ip'] = dataset['ip'].apply(lambda x: encrypt_phone_number(str(x)))

<ipython-input-122-7414705297d1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['ip'] = dataset['ip'].apply(lambda x: encrypt_phone_number(str(x)))


In [ ]:
dataset

,process,keyWordSent,amount,phoneNumber,statusCode,createdAt,canal,country,event,browser,ip,agregator
0,MOBILE,MAQ3,100,X3PwpydKhQfoakULaLsCBRVZw9Z3kMORj7tFyk0pFWg=,INSUFFICIENT_BALANCE,2023-05-11T07:37:38.548Z,Website,BJ,645a225c4744ee6479066772,Light Browser,JHCPyiUh/kDtbQlVO+VObw==,MTNBJ
1,MOBILE,MAQ1,100,X3PwpydwkQPqD28SaLt5BRWeSgKhgdIlW2AAfDjO+yA=,SUCCESS,2023-05-11T08:07:31.532Z,Website,BJ,645a225c4744ee6479066772,Light Browser,JHCPyiUh/kDtbQlVO+VObw==,MTNBJ
2,MOBILE,MAQ2,100,X3PwpydKhQfoakULaLsCBRVZw9Z3kMORj7tFyk0pFWg=,PAYMENT_CANCEL,2023-05-11T08:08:19.453Z,Website,BJ,645a225c4744ee6479066772,Light Browser,JHCPyiUh/kDtbQlVO+VObw==,MTNBJ
3,MOBILE,MAQ1,100,X3PwpydwgQHoMWMSa7t5BXDOet8ySAfdCVG8VpDPEds=,WAITING_CUSTOMER_PAYMENT,2023-05-11T08:10:18.497Z,Website,BJ,645a225c4744ee6479066772,Light Browser,JHCPyiUh/kDtbQlVO+VObw==,FPZ
4,MOBILE,MAQ5,100,X3PwpydKhQfoakULaLsCBRVZw9Z3kMORj7tFyk0pFWg=,PAYMENT_CANCEL,2023-05-11T08:39:40.438Z,Website,BJ,645a225c4744ee6479066772,Light Browser,JHCPyiUh/kDtbQlVO+VObw==,MTNBJ
...,...,...,...,...,...,...,...,...,...,...,...,...
60847,MOBILE,INFG3,1000,X3PwoCRKhQbqH2MQaIEaBYhhcR7ZaUS0sxymair3wnA=,SUCCESS,2024-01-12T11:02:45.493Z,Website,TG,655b1daf01bfb48961045702,"Mozilla/5.0 (Linux., Android 10., K) AppleWebK...",JHCJyiYv5FzuaRdKPuVObw==,FPZ
60848,MOBILE,INFG2,100,X3PwoCRKkQDpMUUVU7sGBfqPmo/FoG+lgSudAqSvBXQ=,WAITING_CUSTOMER_PAYMENT,2024-01-12T11:03:42.488Z,Website,TG,655b1daf01bfb48961045702,"Mozilla/5.0 (Linux., Android 7.1., PLK-TL01H) ...",JHWNyiQh/krudBJVDtdIaQ==,FPZ
60849,CARD,CANDIDATE10,1500,X3P0qydwmRXoMVEWaLseBa49mnoD10/rbyo9snjGYDM=,WAITING_CUSTOMER_PAYMENT,2024-01-12T11:11:52.522Z,Website,TD,64d390d89a201297f90d1c32,Light Browser,JHWPyiAq/kPtaQlTPNBPbg==,FPF
60850,MOBILE,CANDIDATE5,300,X3PwpydKnQToakURU6sCBdyTXmkSLYhYBF3c7KJrGVE=,SUCCESS,2024-01-12T11:20:54.499Z,Website,BJ,64d390d89a201297f90d1c32,"Mozilla/5.0 (Linux., Android 10., K) AppleWebK...",JHOMyiUs5VzubAlVPeNObw==,MTNBJ


In [ ]:
# Cacher les 6 derniers chiffres dans la colonne phoneNumber
dataset['phoneNumber'] = dataset['phoneNumber'].astype(str).str[:-6] + '******'


<ipython-input-107-bf81b92483c2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['phoneNumber'] = dataset['phoneNumber'].astype(str).str[:-6] + '******'


In [ ]:
# Cacher les trois derniers segments dans la colonne ip
dataset['ip'] = dataset['ip'].str.rsplit('.', 3).str[0] + '.***.***.***'

<ipython-input-108-733e9701fa97>:2: FutureWarning: In a future version of pandas all arguments of StringMethods.rsplit except for the argument 'pat' will be keyword-only.
  dataset['ip'] = dataset['ip'].str.rsplit('.', 3).str[0] + '.***.***.***'
<ipython-input-108-733e9701fa97>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['ip'] = dataset['ip'].str.rsplit('.', 3).str[0] + '.***.***.***'


# Analyse exploratoire

In [ ]:
profile = ProfileReport(dataset, title="Pandas Profiling Report")
profile.to_file(output_file="pandas_profiling_report.html")

/usr/local/lib/python3.10/dist-packages/ydata_profiling/profile_report.py:354: UserWarning: Try running command: 'pip install --upgrade Pillow' to avoid ValueError
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/ydata_profiling/utils/dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [35]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60852 entries, 0 to 60851
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   process      60852 non-null  object
 1   amount       60852 non-null  int64 
 2   phoneNumber  60852 non-null  int64 
 3   statusCode   60852 non-null  object
 4   createdAt    60852 non-null  object
 5   canal        60852 non-null  object
 6   country      60852 non-null  object
 7   event        60852 non-null  object
 8   browser      60852 non-null  object
 9   ip           60852 non-null  object
 10  agregator    60852 non-null  object
dtypes: int64(2), object(9)
memory usage: 5.6+ MB


In [36]:
dataset.describe()

,amount,phoneNumber
count,60852.000000,6.085200e+04
mean,3685.742868,9.371022e+10
std,23704.661559,3.622150e+11
min,100.000000,5.383067e+07
25%,100.000000,2.295960e+10
50%,400.000000,2.296772e+10
75%,1000.000000,2.299713e+10
max,500000.000000,8.615523e+12


In [37]:
dataset.isnull().sum()

process        0
amount         0
phoneNumber    0
statusCode     0
createdAt      0
canal          0
country        0
event          0
browser        0
ip             0
agregator      0
dtype: int64

In [38]:
dataset.duplicated().sum()  # 0

16

In [40]:
dataset.drop_duplicates(inplace=True)

<ipython-input-40-1dcc743dc925>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.drop_duplicates(inplace=True)


In [41]:
dataset.duplicated().sum()  # 0

0

In [42]:
dataset

,process,amount,phoneNumber,statusCode,createdAt,canal,country,event,browser,ip,agregator
0,MOBILE,100,22952734444,INSUFFICIENT_BALANCE,2023-05-11T07:37:38.548Z,Website,BJ,645a225c4744ee6479066772,Light Browser,104.28.217.116,MTNBJ
1,MOBILE,100,22967393349,SUCCESS,2023-05-11T08:07:31.532Z,Website,BJ,645a225c4744ee6479066772,Light Browser,104.28.217.116,MTNBJ
2,MOBILE,100,22952734444,PAYMENT_CANCEL,2023-05-11T08:08:19.453Z,Website,BJ,645a225c4744ee6479066772,Light Browser,104.28.217.116,MTNBJ
3,MOBILE,100,22963522309,WAITING_CUSTOMER_PAYMENT,2023-05-11T08:10:18.497Z,Website,BJ,645a225c4744ee6479066772,Light Browser,104.28.217.116,FPZ
4,MOBILE,100,22952734444,PAYMENT_CANCEL,2023-05-11T08:39:40.438Z,Website,BJ,645a225c4744ee6479066772,Light Browser,104.28.217.116,MTNBJ
...,...,...,...,...,...,...,...,...,...,...,...
60847,MOBILE,1000,22892482172,SUCCESS,2024-01-12T11:02:45.493Z,Website,TG,655b1daf01bfb48961045702,"Mozilla/5.0 (Linux., Android 10., K) AppleWebK...",102.164.230.46,FPZ
60848,MOBILE,100,22897264685,WAITING_CUSTOMER_PAYMENT,2024-01-12T11:03:42.488Z,Website,TG,655b1daf01bfb48961045702,"Mozilla/5.0 (Linux., Android 7.1., PLK-TL01H) ...",156.38.82.51,FPZ
60849,CARD,1500,23565929743,WAITING_CUSTOMER_PAYMENT,2024-01-12T11:11:52.522Z,Website,TD,64d390d89a201297f90d1c32,Light Browser,154.73.113.76,FPF
60850,MOBILE,300,22954634294,SUCCESS,2024-01-12T11:20:54.499Z,Website,BJ,64d390d89a201297f90d1c32,"Mozilla/5.0 (Linux., Android 10., K) AppleWebK...",137.255.26.170,MTNBJ


In [43]:
dataset['statusCode'].value_counts()

SUCCESS                      30334
PAYMENT_CANCEL               10710
WAITING_CUSTOMER_PAYMENT      8384
INSUFFICIENT_BALANCE          5718
TRANSACTION_CANCEL            2494
TRANSACTION_CREATED           1410
READY                          632
TRANSACTION_EXPIRED            244
WAITING_PAYMENT_CUSTOMER       241
INTERNAL_PROCESSING_ERROR      196
PAYER_NOT_FOUND                140
NOT_ALLOWED                    125
PAYER_LIMIT_REACHED             81
FAILED                          54
PAYEE_NOT_FOUND                 36
ERROR_AMOUNT_TOO_LOW            15
WAITING_PAYMENT                 14
OPERATOR_UNAVAILABLE             4
INCORRECT_SETTINGS               2
PENDING                          2
Name: statusCode, dtype: int64

In [67]:
#dataset = dataset[dataset['statusCode'] != 'PENDING']
dataset.loc[dataset['statusCode'] == 'PAYER_NOT_FOUND', 'statusCode'] = 'INCORRECT_SETTINGS'

In [71]:
dataset.groupby(['statusCode']).size()

statusCode
INCORRECT_SETTINGS            653
INSUFFICIENT_BALANCE         5718
SUCCESS                     30334
TRANSACTION_CANCEL          13448
WAITING_CUSTOMER_PAYMENT    10683
dtype: int64

In [72]:
dataset.to_excel('/content/drive/Othercomputers/Mon ordinateur/DIT MASTER1 2022/MASTER 2/MEMOIRE/data/data/dataset_cleaned_statusCode_12022024.xlsx', index=False)

In [83]:
from sklearn.model_selection import train_test_split

# Supposons que X contient vos variables explicatives et y votre variable cible
X = dataset  # Variables explicatives
y = dataset['statusCode']  # Variable cible

# Séparation en train (70%), test (20%) et validation (10%) en fonction de y (statusCode)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42, stratify=y_temp)

# Affichage des tailles des ensembles
print("Taille de l'ensemble d'entraînement (train) :", X_train.shape[0])
print("Taille de l'ensemble de test :", X_test.shape[0])
print("Taille de l'ensemble de validation :", X_val.shape[0])


Taille de l'ensemble d'entraînement (train) : 42585
Taille de l'ensemble de test : 12228
Taille de l'ensemble de validation : 6023


In [82]:
y_train

27947        INSUFFICIENT_BALANCE
55618        INSUFFICIENT_BALANCE
46990                     SUCCESS
24432    WAITING_CUSTOMER_PAYMENT
32672          TRANSACTION_CANCEL
                   ...           
5510                      SUCCESS
16372                     SUCCESS
52156        INSUFFICIENT_BALANCE
24382    WAITING_CUSTOMER_PAYMENT
39219        INSUFFICIENT_BALANCE
Name: statusCode, Length: 42585, dtype: object

In [84]:
import pandas as pd

# Créer un DataFrame avec les occurrences de statusCode dans chaque ensemble
occurrences_train = X_train['statusCode'].value_counts()
occurrences_test = X_test['statusCode'].value_counts()
occurrences_val = X_val['statusCode'].value_counts()

# Créer un tableau pour chaque ensemble
train_table = pd.DataFrame({'StatusCode': occurrences_train.index, 'Occurrences (Train)': occurrences_train.values})
test_table = pd.DataFrame({'StatusCode': occurrences_test.index, 'Occurrences (Test)': occurrences_test.values})
val_table = pd.DataFrame({'StatusCode': occurrences_val.index, 'Occurrences (Validation)': occurrences_val.values})

# Fusionner les tableaux en un seul tableau en utilisant la colonne StatusCode comme index
occurrences_table = train_table.merge(test_table, on='StatusCode', how='outer').merge(val_table, on='StatusCode', how='outer')
occurrences_table = occurrences_table.set_index('StatusCode')

# Remplacer les valeurs NaN par 0
occurrences_table = occurrences_table.fillna(0).astype(int)

# Afficher le tableau des occurrences
print(occurrences_table)



                          Occurrences (Train)  Occurrences (Test)  \
StatusCode                                                          
SUCCESS                                 21234                6097   
TRANSACTION_CANCEL                       9413                2704   
WAITING_CUSTOMER_PAYMENT                 7478                2147   
INSUFFICIENT_BALANCE                     4003                1149   
INCORRECT_SETTINGS                        457                 131   

                          Occurrences (Validation)  
StatusCode                                          
SUCCESS                                       3003  
TRANSACTION_CANCEL                            1331  
WAITING_CUSTOMER_PAYMENT                      1058  
INSUFFICIENT_BALANCE                           566  
INCORRECT_SETTINGS                              65  


In [85]:
# prompt: génère moi la sortie précédente en latex

print(occurrences_table.to_latex())


\begin{tabular}{lrrr}
\toprule
{} &  Occurrences (Train) &  Occurrences (Test) &  Occurrences (Validation) \\
StatusCode               &                      &                     &                           \\
\midrule
SUCCESS                  &                21234 &                6097 &                      3003 \\
TRANSACTION\_CANCEL       &                 9413 &                2704 &                      1331 \\
WAITING\_CUSTOMER\_PAYMENT &                 7478 &                2147 &                      1058 \\
INSUFFICIENT\_BALANCE     &                 4003 &                1149 &                       566 \\
INCORRECT\_SETTINGS       &                  457 &                 131 &                        65 \\
\bottomrule
\end{tabular}



<ipython-input-85-fd580d5a7b6e>:3: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(occurrences_table.to_latex())
